## Simulate data

This script is used to generate simulated data. Then we can run parameter_recovery to estimate those parameters from the simulated data


In [40]:
# Built-in/Generic Imports
import os

# Libs
import numpy as np
import pandas as pd
from scipy.stats import bernoulli

In [41]:

### Defining the grids, and functions to make sets as dataframes: choice_set,param_set,response_set

def set_grids(nb_kappa=10,nb_gamma=10):
    grid_design = {
        # now: time_null = 0
        'cdd_immed_wait': [0],
        # time to get reward (delay_wait)
        'cdd_delay_wait': [2, 7, 15, 36, 72, 108, 144, 180],
        # immediate reward from experiment: $2, $10, $20 ... fixed to $10
        'cdd_immed_amt': [10, 20],
        # reward amount set to vary according to experiment
        'cdd_delay_amt': [7, 14, 21, 30, 41, 50, 65]
    }
    grid_param = {
        # 50 points on [10^-5, ..., 1] in a log scale
        'kappa_gt': np.logspace(-5, 0, nb_kappa, base=10),
        # 10 points on (0, 5] in a linear scale
        'gamma_gt': np.linspace(0, 5, nb_gamma+1)[1:]
    }
    return grid_design,grid_param

def insert_var(grid=[],var_nb=0,list_var=[]):
    if not var_nb:
        grid = list_var
    elif var_nb==1:
        grid = [[g]+[i] for g in grid for i in list_var]
    else:
        grid = [g+[i] for g in grid for i in list_var]
    return grid

def make_grid(grid_values):
    grid = []
    labels = grid_values.keys()
    for var_nb,row in enumerate(grid_values.items()):
        grid = insert_var(grid=grid,var_nb=var_nb,list_var=row[1])
    grid_df = pd.DataFrame(grid,columns=labels)
    return grid_df


In [42]:

def simulate_response(design,PARAM_TRUE):
    # Calculate the probability to choose a variable option
    tn, tr, vn, vr = (design['cdd_immed_wait'], design['cdd_delay_wait'],design['cdd_immed_amt'], design['cdd_delay_amt'])
    kappa, gamma = PARAM_TRUE['kappa_gt'], PARAM_TRUE['gamma_gt']
    
    SV_null = vn * (1. / (1 + kappa * tn))
    SV_reward = vr * (1. / (1 + kappa * tr))
    p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))

    # Randomly sample a binary choice response from Bernoulli distribution
    return bernoulli.rvs(p_obs)

def generate_response(design_df,PARAM_TRUE):
    response= []
    df = pd.DataFrame([])
    for index, design in design_df.iterrows():
        response += [simulate_response(design,PARAM_TRUE)]
    df['response'] = response
    return df


In [43]:
# can set resolution (number of parameter) for kappa and gamma
grid_design,grid_param = set_grids(nb_kappa=3,nb_gamma=3)

design_df = make_grid(grid_design)
fn = os.path.join('simul','design_set.csv')
print('Saving design to : {}'.format(fn))
design_df.to_csv(fn)

param_df = make_grid(grid_param)
fn = os.path.join('simul','ground_truth.csv')
print('Saving ground truth parameters to : {}'.format(fn))
param_df.to_csv(fn)

for index, PARAM_TRUE in param_df.iterrows():
    response_df = generate_response(design_df,PARAM_TRUE)
    fn = os.path.join('simul','response','p{0:04d}.csv'.format(index))
    print('Saving response to : {}'.format(fn))
    response_df.to_csv(fn)

Saving design to : simul/design_set.csv
Saving ground truth parameters to : simul/ground_truth.csv
Saving response to : simul/response/p0000.csv
Saving response to : simul/response/p0001.csv
Saving response to : simul/response/p0002.csv
Saving response to : simul/response/p0003.csv
Saving response to : simul/response/p0004.csv
Saving response to : simul/response/p0005.csv
Saving response to : simul/response/p0006.csv
Saving response to : simul/response/p0007.csv
Saving response to : simul/response/p0008.csv
